In [ ]:
# example code to load MetaProD output dataset_peptides_initial.tsv
# it will then run feature selection and some ML models
# it will finally generate ROC curves

# some variables can be changed in a section denoted below
# but some of the code has been hard coded for these data sets and phenotypes
#  and would need to be changed for other data sets

# the cd data sets can be used as an example when there are more than 2 phenotypes

# the t1d/al1 data sets can be used as an example when there are 2 phenotypes

import pandas as pd
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVC
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import kaleido
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.io as pio
from sklearn.pipeline import make_pipeline
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from numpy import interp
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
import plotly
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.preprocessing import FunctionTransformer
import kaleido
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score
import plotly.io as pio
import time
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2

##############################################################
##############################################################
##### section where variables need to be changed

# datasets to analyze
# MetaProD will name the outputs as dataset_peptides_initial.tsv
datasets = ["cd2",]

# should not need to change any of these but the colors are used in the graphs
labels = ["Human", "Bacterial", "Combined", "Sequence-Only"]
colors = ["Green", "Red", "Blue", "Purple"]

# data where the MetaProD output is stored
# this is the directory with the dataset_peptides_initial.tsv files
input_dir = "z:/ml_project/example/data"

# directory to save output
output_dir = "z:/ml_project/example/figures/"

# number of splits and repeats for ML processing
n_splits = 5
n_repeats = 10

# some of the code below is hard coded for the specific data sets used and may need
# to be changed for other data sets
# in particular, the code was hard coded to mention the data sets and phenotypes in the data

##############################################################
##############################################################

if not os.path.exists(input_dir):
    os.makedirs(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
           
def log_transform(x):
    # add one because imputation could be 0
    return np.log2(x + 1)

def median_center(x):
    x.update(x.sub(x.median(axis=0),axis=1))
    return(x)

def new_rf_rfe_(x, y):
    print(x, y)
    
transformer = FunctionTransformer(log_transform)
center = FunctionTransformer(median_center)
new_rf_rfe = FunctionTransformer(new_rf_rfe_)

cv = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=12)
si = SimpleImputer(strategy='constant', fill_value=0)  
scaler = RobustScaler()

models = [RandomForestClassifier(n_jobs=-1), svm.SVC(probability=True), KNeighborsClassifier(n_jobs=-1)]

descriptions = ["RF", "SVM", "KNN"]

rfes = [None]

rfes = [None, 
        RFECV(estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1, step=0.05),
        SelectFromModel(LinearSVC(penalty="l2", max_iter=10000, dual="auto")),
        SelectKBest()]

rfe_desc = ["None", "RF-RFECV", "LinearSVC", "SelectKBest"]

rows_list_pep = []
rows_list_pro = []

def generate_graph(features, classes, dataset, phenotype, source, multi):
    k = 0
    for m in models:
        print("Model: %s" % descriptions[k])
        l = 0
        for rfe in rfes:
            print("RFE method: %s" % rfe_desc[l])
            if rfe == None:
                pipeline = Pipeline(steps=[('i', si), ('scaler', scaler), ('m', m)])
            else:
                pipeline = Pipeline(steps=[('i', si), ('scaler', scaler), ('rfe', rfe), ('m', m)]) 

            mean_fpr = np.linspace(0,1,100)
            ###
            fig = go.Figure()
            fig.add_shape(
                type='line', line=dict(dash='dash'),
                x0=0, x1=1, y0=0, y1=1
            )

            for i in range(len(features)):
                st = time.time()
                dict1 = {}
                tprs = []
                aucs = []
                num_features = [] 
                runtime = 0
                for j, (train_index, test_index) in enumerate(cv.split(features[i], classes[i])):
                #for train_index, test_index in cv.split(features[i]):
                    X_train, X_test = features[i].iloc[train_index], features[i].iloc[test_index]
                    y_train, y_test = classes[i].iloc[train_index], classes[i].iloc[test_index]
                    #X_train, X_test = features[i].iloc[train_index], features[i].iloc[test_index]
                    #y_train, y_test = classes[i].iloc[train_index], classes[i].iloc[test_index]
                    if multi == True:
                        fpr, tpr, roc_auc = dict(), dict(), dict()
                    lb = LabelBinarizer().fit(y_train)
                    lb_test = lb.transform(y_test)
                    pp = pipeline.fit(X_train, y_train)
                    num_features.append(pp['m'].n_features_in_)
                    pred = pp.predict_proba(X_test)
                    #
                    if multi == True:
                        fpr['micro'], tpr['micro'], _ = roc_curve(lb_test.ravel(), pred.ravel())
                        roc_auc['micro'] = auc(fpr['micro'], tpr['micro'])
                        tprs.append(interp(mean_fpr, fpr['micro'], tpr['micro']))
                        aucs.append(roc_auc['micro'])
                    else:
                        fpr, tpr, t = roc_curve(lb_test, pred[:, 1])
                        roc_auc = auc(fpr, tpr)
                        tprs.append(interp(mean_fpr, fpr, tpr))
                        aucs.append(roc_auc)
    
                mean_tpr = np.mean(tprs, axis=0)
                mean_auc = auc(mean_fpr, mean_tpr)
                std_auc = np.std(aucs)
                mean_features = np.mean(num_features)
                et = time.time()
                runtime = et - st
                dict1.update({'dataset':dataset, 'phenotype':phenotype, 'fs':rfe_desc[l], 'model':descriptions[k], 'label': labels[i], 'score':round(mean_auc, 3),
                              'std':round(std_auc, 3), '# features used':round(mean_features, 0), 'runtime': round(runtime, 3), 'samples':features[i].shape[0],
                              'features':features[i].shape[1]})
                print(dict1)
                if source == 'proteins':
                    rows_list_pro.append(dict1)
                else:
                    rows_list_pep.append(dict1)
                fig.add_trace(go.Scatter(x=mean_fpr, y=mean_tpr, name=r'%s (AUC = %0.2f &plusmn; %0.2f)' % (labels[i], mean_auc, std_auc), mode='lines', line=dict(color=colors[i])))

            fig.update_layout(
                xaxis_title='FPR',
                yaxis_title='TPR',
                yaxis=dict(scaleanchor="x", scaleratio=1),
                xaxis=dict(constrain='domain'),
                width=600, height=600
            )
            fig.update_layout(
                go.Layout(plot_bgcolor="white", margin_l=5, margin_t=5, margin_b=5, margin_r=5,
                          font=dict(family="Serif", size=22)), showlegend=True,
                          legend=dict(yanchor="bottom", xanchor="right", y=0, x=1, orientation="v", traceorder='normal'),
            )
            fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
            fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
            # fig.show()
    
            fig2=px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
            if source == 'proteins':
                fig2.write_image(os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats_proteins.pdf" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                fig2.write_image(os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats_proteins.eps" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                time.sleep(2)

                pio.write_image(fig, os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats_proteins.pdf" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                pio.write_image(fig, os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats_proteins.eps" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                ###
            elif source == 'peptides':
                fig2.write_image(os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats.pdf" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                fig2.write_image(os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats.eps" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                time.sleep(2)

                pio.write_image(fig, os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats.pdf" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                pio.write_image(fig, os.path.join(output_dir, "%s_%s_%s_%s_%ssplits_%srepeats.eps" % (dataset, phenotype, rfe_desc[l], descriptions[k], n_splits, n_repeats)), engine="kaleido", scale=2)
                ###                
            l += 1
            
        k += 1
                 
def do_ml(d):
    print("##########################################################################")
    print("##########################################################################")
    print("Dataset: %s" % d)  

    df_pep = pd.read_csv(os.path.join(input_dir, "%s_peptides_initial.tsv" % d), sep='\t')
    df_pep_cols = df_pep.iloc[:,:6]
    
    # here we just consider sequence only without regard to species/protein
    # sometimes a given peptide sequence maps to a different species/protein in a sample but we don't care about that here
    df_pep_sequence = df_pep.groupby(['sequence'], dropna=False).median(numeric_only=True)
    df_pep.replace(0, np.nan, inplace=True)
    df_pep_sequence.replace(0, np.nan, inplace=True)
    
    df_pep_pa = df_pep[df_pep.columns[df_pep.columns.str.contains("Peak Area")]].copy()
    df_pep_pa_sequence = df_pep_sequence[df_pep_sequence.columns[df_pep_sequence.columns.str.contains("Peak Area")]].copy()
    
    # remove peptides that aren't in 50% of the samples
    if d == 'cd2':
        df_pep_pa.dropna(axis=0, thresh=int(0.60*df_pep_pa.shape[1]), inplace=True)
        df_pep_pa_sequence.dropna(axis=0, thresh=int(0.60*df_pep_pa_sequence.shape[1]), inplace=True)
    else:
        df_pep_pa.dropna(axis=0, thresh=int(0.75*df_pep_pa.shape[1]), inplace=True)
        df_pep_pa_sequence.dropna(axis=0, thresh=int(0.75*df_pep_pa_sequence.shape[1]), inplace=True)
        
    df_pep_final = df_pep_cols.join(df_pep_pa, how="right")
    df_pep_final_pro = df_pep_final.copy()
    df_pep_final_pro.drop(columns=['sequence'], inplace=True)    
    df_data_proteins = df_pep_final_pro.groupby(['accession', 'gene', 'description', 'ppid', 'organism'], dropna=False).median(numeric_only=False)
    df_pep_final = df_pep_final.set_index(['sequence', 'accession', 'gene', 'description', 'ppid', 'organism'])
     
    df_pep_final_human = df_pep_final[df_pep_final.index.get_level_values('ppid').str.contains('UP000005640')].T
    df_pep_final_bac = df_pep_final[~df_pep_final.index.get_level_values('ppid').str.contains('UP000005640')].T
    df_pep_final_combined = df_pep_final.T
    df_pep_final_sequence = df_pep_pa_sequence.T

    df_pro_final_human = df_data_proteins[df_data_proteins.index.get_level_values('ppid').str.contains('UP000005640')].T
    df_pro_final_bac = df_data_proteins[~df_data_proteins.index.get_level_values('ppid').str.contains('UP000005640')].T
    df_pro_final_combined = df_data_proteins.T    
    
    if d == 'cd2' or d == 'cd3' or d == 'cd4':
        p_cd_c = ('CD', 'Control')
        p_uc_c = ('UC', 'Control')
        p_cd_uc = ('CD', 'UC')
        p_cd_uc_c = ('CD', 'UC', 'Control')
        df_pep_final_human_cd_c = df_pep_final_human[df_pep_final_human.index.str.endswith(p_cd_c)]
        df_pep_final_human_uc_c = df_pep_final_human[df_pep_final_human.index.str.endswith(p_uc_c)]
        df_pep_final_human_cd_uc = df_pep_final_human[df_pep_final_human.index.str.endswith(p_cd_uc)]
        df_pep_final_human_cd_uc_c = df_pep_final_human[df_pep_final_human.index.str.endswith(p_cd_uc_c)]
        
        df_pep_final_bac_cd_c = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_cd_c)]
        df_pep_final_bac_uc_c = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_uc_c)]
        df_pep_final_bac_cd_uc = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_cd_uc)]
        df_pep_final_bac_cd_uc_c = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_cd_uc_c)]
        
        df_pep_final_combined_cd_c = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_cd_c)]
        df_pep_final_combined_uc_c = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_uc_c)]
        df_pep_final_combined_cd_uc = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_cd_uc)]
        df_pep_final_combined_cd_uc_c = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_cd_uc_c)]

        df_pep_final_sequence_cd_c = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith(p_cd_c)]
        df_pep_final_sequence_uc_c = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith((p_uc_c))]
        df_pep_final_sequence_cd_uc = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith(p_cd_uc)]
        df_pep_final_sequence_cd_uc_c = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith(p_cd_uc_c)]

        # pro
        df_pro_final_human_cd_c = df_pro_final_human[df_pro_final_human.index.str.endswith(p_cd_c)]
        df_pro_final_human_uc_c = df_pro_final_human[df_pro_final_human.index.str.endswith(p_uc_c)]
        df_pro_final_human_cd_uc = df_pro_final_human[df_pro_final_human.index.str.endswith(p_cd_uc)]
        df_pro_final_human_cd_uc_c = df_pro_final_human[df_pro_final_human.index.str.endswith(p_cd_uc_c)]
    
        df_pro_final_bac_cd_c = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_cd_c)]
        df_pro_final_bac_uc_c = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_uc_c)]
        df_pro_final_bac_cd_uc = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_cd_uc)]
        df_pro_final_bac_cd_uc_c = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_cd_uc_c)]
    
        df_pro_final_combined_cd_c = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_cd_c)]
        df_pro_final_combined_uc_c = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_uc_c)]
        df_pro_final_combined_cd_uc = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_cd_uc)]
        df_pro_final_combined_cd_uc_c = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_cd_uc_c)]
        
        df_pep_final_human_classes_cd_c = []
        for index, row in df_pep_final_human_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_human_classes_cd_c.append('CD')
            else:
                df_pep_final_human_classes_cd_c.append('Control')
        df_pep_final_human_classes_cd_c = pd.Series(df_pep_final_human_classes_cd_c)
        
        df_pep_final_human_classes_uc_c = []
        for index, row in df_pep_final_human_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pep_final_human_classes_uc_c.append('UC')
            else:
                df_pep_final_human_classes_uc_c.append('Control')
        df_pep_final_human_classes_uc_c = pd.Series(df_pep_final_human_classes_uc_c)
        
        df_pep_final_human_classes_cd_uc = []
        for index, row in df_pep_final_human_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_human_classes_cd_uc.append('CD')
            else:
                df_pep_final_human_classes_cd_uc.append('UC')
        df_pep_final_human_classes_cd_uc = pd.Series(df_pep_final_human_classes_cd_uc)

        df_pep_final_human_classes_cd_uc_c = []
        for index, row in df_pep_final_human_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_human_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pep_final_human_classes_cd_uc_c.append('UC')
            else:
                df_pep_final_human_classes_cd_uc_c.append('Control')
        df_pep_final_human_classes_cd_uc_c = pd.Series(df_pep_final_human_classes_cd_uc_c)
        
        df_pep_final_bac_classes_cd_c = []
        for index, row in df_pep_final_bac_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_bac_classes_cd_c.append('CD')
            else:
                df_pep_final_bac_classes_cd_c.append('Control')
        df_pep_final_bac_classes_cd_c = pd.Series(df_pep_final_bac_classes_cd_c)
        
        df_pep_final_bac_classes_uc_c = []
        for index, row in df_pep_final_bac_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pep_final_bac_classes_uc_c.append('UC')
            else:
                df_pep_final_bac_classes_uc_c.append('Control')
        df_pep_final_bac_classes_uc_c = pd.Series(df_pep_final_bac_classes_uc_c)
        
        df_pep_final_bac_classes_cd_uc = []
        for index, row in df_pep_final_bac_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_bac_classes_cd_uc.append('CD')
            else:
                df_pep_final_bac_classes_cd_uc.append('UC')
        df_pep_final_bac_classes_cd_uc = pd.Series(df_pep_final_bac_classes_cd_uc)   

        df_pep_final_bac_classes_cd_uc_c = []
        for index, row in df_pep_final_bac_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_bac_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pep_final_bac_classes_cd_uc_c.append('UC')
            else:
                df_pep_final_bac_classes_cd_uc_c.append('Control')
        df_pep_final_bac_classes_cd_uc_c = pd.Series(df_pep_final_bac_classes_cd_uc_c)
        
        # combined
        df_pep_final_combined_classes_cd_c = []
        for index, row in df_pep_final_combined_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_combined_classes_cd_c.append('CD')
            else:
                df_pep_final_combined_classes_cd_c.append('Control')
        df_pep_final_combined_classes_cd_c = pd.Series(df_pep_final_combined_classes_cd_c)
                
        df_pep_final_combined_classes_uc_c = []
        for index, row in df_pep_final_combined_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pep_final_combined_classes_uc_c.append('UC')
            else:
                df_pep_final_combined_classes_uc_c.append('Control')
        df_pep_final_combined_classes_uc_c = pd.Series(df_pep_final_combined_classes_uc_c)
        
        df_pep_final_combined_classes_cd_uc = []
        for index, row in df_pep_final_combined_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_combined_classes_cd_uc.append('CD')
            else:
                df_pep_final_combined_classes_cd_uc.append('Control')
        df_pep_final_combined_classes_cd_uc = pd.Series(df_pep_final_combined_classes_cd_uc)       

        df_pep_final_combined_classes_cd_uc_c = []
        for index, row in df_pep_final_combined_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_combined_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pep_final_combined_classes_cd_uc_c.append('UC')
            else:
                df_pep_final_combined_classes_cd_uc_c.append('Control')
        df_pep_final_combined_classes_cd_uc_c = pd.Series(df_pep_final_combined_classes_cd_uc_c)
        
        # sequence-only, which is sequence only without regard to species/protein
        df_pep_final_sequence_classes_cd_c = []
        for index, row in df_pep_final_sequence_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_sequence_classes_cd_c.append('CD')
            else:
                df_pep_final_sequence_classes_cd_c.append('Control')
        df_pep_final_sequence_classes_cd_c = pd.Series(df_pep_final_sequence_classes_cd_c)

        df_pep_final_sequence_classes_uc_c = []
        for index, row in df_pep_final_sequence_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pep_final_sequence_classes_uc_c.append('UC')
            else:
                df_pep_final_sequence_classes_uc_c.append('Control')
        df_pep_final_sequence_classes_uc_c = pd.Series(df_pep_final_sequence_classes_uc_c)
                
        df_pep_final_sequence_classes_cd_uc = []
        for index, row in df_pep_final_sequence_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_sequence_classes_cd_uc.append('CD')
            else:
                df_pep_final_sequence_classes_cd_uc.append('UC')
        df_pep_final_sequence_classes_cd_uc = pd.Series(df_pep_final_sequence_classes_cd_uc)

        df_pep_final_sequence_classes_cd_uc_c = []
        for index, row in df_pep_final_sequence_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pep_final_sequence_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pep_final_sequence_classes_cd_uc_c.append('UC')
            else:
                df_pep_final_sequence_classes_cd_uc_c.append('Control')
        df_pep_final_sequence_classes_cd_uc_c = pd.Series(df_pep_final_sequence_classes_cd_uc_c)
        
        # pro
        df_pro_final_human_classes_cd_c = []
        for index, row in df_pro_final_human_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_human_classes_cd_c.append('CD')
            else:
                df_pro_final_human_classes_cd_c.append('Control')
        df_pro_final_human_classes_cd_c = pd.Series(df_pro_final_human_classes_cd_c)
        
        df_pro_final_human_classes_uc_c = []
        for index, row in df_pro_final_human_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pro_final_human_classes_uc_c.append('UC')
            else:
                df_pro_final_human_classes_uc_c.append('Control')
        df_pro_final_human_classes_uc_c = pd.Series(df_pro_final_human_classes_uc_c)
        
        df_pro_final_human_classes_cd_uc = []
        for index, row in df_pro_final_human_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_human_classes_cd_uc.append('CD')
            else:
                df_pro_final_human_classes_cd_uc.append('UC')
        df_pro_final_human_classes_cd_uc = pd.Series(df_pro_final_human_classes_cd_uc)

        df_pro_final_human_classes_cd_uc_c = []
        for index, row in df_pro_final_human_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_human_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pro_final_human_classes_cd_uc_c.append('UC')
            else:
                df_pro_final_human_classes_cd_uc_c.append('Control')
        df_pro_final_human_classes_cd_uc_c = pd.Series(df_pro_final_human_classes_cd_uc_c)
        
        df_pro_final_bac_classes_cd_c = []
        for index, row in df_pro_final_bac_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_bac_classes_cd_c.append('CD')
            else:
                df_pro_final_bac_classes_cd_c.append('Control')
        df_pro_final_bac_classes_cd_c = pd.Series(df_pro_final_bac_classes_cd_c)
        
        df_pro_final_bac_classes_uc_c = []
        for index, row in df_pro_final_bac_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pro_final_bac_classes_uc_c.append('UC')
            else:
                df_pro_final_bac_classes_uc_c.append('Control')
        df_pro_final_bac_classes_uc_c = pd.Series(df_pro_final_bac_classes_uc_c)
        
        df_pro_final_bac_classes_cd_uc = []
        for index, row in df_pro_final_bac_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_bac_classes_cd_uc.append('CD')
            else:
                df_pro_final_bac_classes_cd_uc.append('UC')
        df_pro_final_bac_classes_cd_uc = pd.Series(df_pro_final_bac_classes_cd_uc)   

        df_pro_final_bac_classes_cd_uc_c = []
        for index, row in df_pro_final_bac_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_bac_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pro_final_bac_classes_cd_uc_c.append('UC')
            else:
                df_pro_final_bac_classes_cd_uc_c.append('Control')
        df_pro_final_bac_classes_cd_uc_c = pd.Series(df_pro_final_bac_classes_cd_uc_c)
        
        # combined
        df_pro_final_combined_classes_cd_c = []
        for index, row in df_pro_final_combined_cd_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_combined_classes_cd_c.append('CD')
            else:
                df_pro_final_combined_classes_cd_c.append('Control')
        df_pro_final_combined_classes_cd_c = pd.Series(df_pro_final_combined_classes_cd_c)
                
        df_pro_final_combined_classes_uc_c = []
        for index, row in df_pro_final_combined_uc_c.iterrows():
            if str(index).endswith('UC'):
                df_pro_final_combined_classes_uc_c.append('UC')
            else:
                df_pro_final_combined_classes_uc_c.append('Control')
        df_pro_final_combined_classes_uc_c = pd.Series(df_pro_final_combined_classes_uc_c)
                
        df_pro_final_combined_classes_cd_uc = []
        for index, row in df_pro_final_combined_cd_uc.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_combined_classes_cd_uc.append('CD')
            else:
                df_pro_final_combined_classes_cd_uc.append('UC')
        df_pro_final_combined_classes_cd_uc = pd.Series(df_pro_final_combined_classes_cd_uc)       

        df_pro_final_combined_classes_cd_uc_c = []
        for index, row in df_pro_final_combined_cd_uc_c.iterrows():
            if str(index).endswith('CD'):
                df_pro_final_combined_classes_cd_uc_c.append('CD')
            elif str(index).endswith('UC'):
                df_pro_final_combined_classes_cd_uc_c.append('UC')
            else:
                df_pro_final_combined_classes_cd_uc_c.append('Control')
        df_pro_final_combined_classes_cd_uc_c = pd.Series(df_pro_final_combined_classes_cd_uc_c)

        print("Peptides")
        print("Human CD C has %s samples and %s peptides." % (df_pep_final_human_cd_c.shape[0], df_pep_final_human_cd_c.shape[1]))
        print("Bac CD C has %s samples and %s peptides." % (df_pep_final_bac_cd_c.shape[0], df_pep_final_bac_cd_c.shape[1]))       
        print("Combined CD C has %s samples and %s peptides." % (df_pep_final_combined_cd_c.shape[0], df_pep_final_combined_cd_c.shape[1]))       
        print("Sequence-Only CD C has %s samples and %s peptides." % (df_pep_final_sequence_cd_c.shape[0], df_pep_final_sequence_cd_c.shape[1]))   
        print("---")
        print("Human UC C has %s samples and %s peptides." % (df_pep_final_human_uc_c.shape[0], df_pep_final_human_uc_c.shape[1]))
        print("Bac UC C has %s samples and %s peptides." % (df_pep_final_bac_uc_c.shape[0], df_pep_final_bac_uc_c.shape[1]))       
        print("Combined UC C has %s samples and %s peptides." % (df_pep_final_combined_uc_c.shape[0], df_pep_final_combined_uc_c.shape[1]))       
        print("Sequence-Only UC C has %s samples and %s peptides." % (df_pep_final_sequence_uc_c.shape[0], df_pep_final_sequence_uc_c.shape[1]))   
        print("---")
        print("Human CD UC has %s samples and %s peptides." % (df_pep_final_human_cd_uc.shape[0], df_pep_final_human_cd_uc.shape[1]))
        print("Bac CD UC has %s samples and %s peptides." % (df_pep_final_bac_cd_uc.shape[0], df_pep_final_bac_cd_uc.shape[1]))       
        print("Combined CD UC has %s samples and %s peptides." % (df_pep_final_combined_cd_uc.shape[0], df_pep_final_combined_cd_uc.shape[1]))       
        print("Sequence-Only CD UC has %s samples and %s peptides." % (df_pep_final_sequence_cd_uc.shape[0], df_pep_final_sequence_cd_uc.shape[1]))   
        print("---")
        print("Human CD UC C has %s samples and %s peptides." % (df_pep_final_human_cd_uc_c.shape[0], df_pep_final_human_cd_uc_c.shape[1]))
        print("Bac CD UC C has %s samples and %s peptides." % (df_pep_final_bac_cd_uc_c.shape[0], df_pep_final_bac_cd_uc_c.shape[1]))       
        print("Combined CD UC C has %s samples and %s peptides." % (df_pep_final_combined_cd_uc_c.shape[0], df_pep_final_combined_cd_uc_c.shape[1]))       
        print("Sequence-Only CD UC C has %s samples and %s peptides." % (df_pep_final_sequence_cd_uc_c.shape[0], df_pep_final_sequence_cd_uc_c.shape[1]))   

        print("Proteins")
        # pro 
        print("Human CD C has %s samples and %s proteins." % (df_pro_final_human_cd_c.shape[0], df_pro_final_human_cd_c.shape[1]))
        print("Bac CD C has %s samples and %s proteins." % (df_pro_final_bac_cd_c.shape[0], df_pro_final_bac_cd_c.shape[1]))       
        print("Combined CD C has %s samples and %s proteins." % (df_pro_final_combined_cd_c.shape[0], df_pro_final_combined_cd_c.shape[1]))       
        print("---")
        print("Human UC C has %s samples and %s proteins." % (df_pro_final_human_uc_c.shape[0], df_pro_final_human_uc_c.shape[1]))
        print("Bac UC C has %s samples and %s proteins." % (df_pro_final_bac_uc_c.shape[0], df_pro_final_bac_uc_c.shape[1]))       
        print("Combined UC C has %s samples and %s proteins." % (df_pro_final_combined_uc_c.shape[0], df_pro_final_combined_uc_c.shape[1]))        
        print("---")
        print("Human CD UC has %s samples and %s proteins." % (df_pro_final_human_cd_uc.shape[0], df_pro_final_human_cd_uc.shape[1]))
        print("Bac CD UC has %s samples and %s proteins." % (df_pro_final_bac_cd_uc.shape[0], df_pro_final_bac_cd_uc.shape[1]))       
        print("Combined CD UC has %s samples and %s proteins." % (df_pro_final_combined_cd_uc.shape[0], df_pro_final_combined_cd_uc.shape[1]))         
        print("---")
        print("Human CD UC C has %s samples and %s proteins." % (df_pro_final_human_cd_uc_c.shape[0], df_pro_final_human_cd_uc_c.shape[1]))
        print("Bac CD UC C has %s samples and %s proteins." % (df_pro_final_bac_cd_uc_c.shape[0], df_pro_final_bac_cd_uc_c.shape[1]))       
        print("Combined CD UC C has %s samples and %s proteins." % (df_pro_final_combined_cd_uc_c.shape[0], df_pro_final_combined_cd_uc_c.shape[1]))   
        
        print("##########################################################################")
        print("Peptides")
        print("##########################################################################")
        print("CD vs UC vs C")
        features = [df_pep_final_human_cd_uc_c, df_pep_final_bac_cd_uc_c, df_pep_final_combined_cd_uc_c, df_pep_final_sequence_cd_uc_c]
        classes = [df_pep_final_human_classes_cd_uc_c, df_pep_final_bac_classes_cd_uc_c, df_pep_final_combined_classes_cd_uc_c, df_pep_final_sequence_classes_cd_uc_c]
        generate_graph(features, classes, d, "CD-UC-C", "peptides", True)
        
        print("##########################################################################")
        print("Proteins")
        print("##########################################################################")
        print("CD vs UC vs C")
        features = [df_pro_final_human_cd_uc_c, df_pro_final_bac_cd_uc_c, df_pro_final_combined_cd_uc_c]
        classes = [df_pro_final_human_classes_cd_uc_c, df_pro_final_bac_classes_cd_uc_c, df_pro_final_combined_classes_cd_uc_c]
        generate_graph(features, classes, d, "CD-UC-C", "proteins", True)
        
    # t1d has new onset (NO), seropositive (SP), seronegative (SN), control (Control)
    elif d == 't1d1':
        p_sp_c = ('SP', 'Control')        
        df_pep_final_human_t1d_c = df_pep_final_human[df_pep_final_human.index.str.endswith(p_sp_c)]
  
        df_pep_final_bac_t1d_c = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_sp_c)]
    
        df_pep_final_combined_t1d_c = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_sp_c)]

        df_pep_final_sequence_t1d_c = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith(p_sp_c)]

        df_pro_final_human_t1d_c = df_pro_final_human[df_pro_final_human.index.str.endswith(p_sp_c)]
  
        df_pro_final_bac_t1d_c = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_sp_c)]
    
        df_pro_final_combined_t1d_c = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_sp_c)]
        
        df_pep_final_human_classes_t1d_c = []
        for index, row in df_pep_final_human_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pep_final_human_classes_t1d_c.append('SP')
            else:
                df_pep_final_human_classes_t1d_c.append('Control')
        df_pep_final_human_classes_t1d_c = pd.Series(df_pep_final_human_classes_t1d_c)   
        
        df_pep_final_bac_classes_t1d_c = []
        for index, row in df_pep_final_bac_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pep_final_bac_classes_t1d_c.append('SP')
            else:
                df_pep_final_bac_classes_t1d_c.append('Control')
        df_pep_final_bac_classes_t1d_c = pd.Series(df_pep_final_bac_classes_t1d_c)   
                      
        # combined
        df_pep_final_combined_classes_t1d_c = []
        for index, row in df_pep_final_combined_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pep_final_combined_classes_t1d_c.append('SP')
            else:
                df_pep_final_combined_classes_t1d_c.append('Control')
        df_pep_final_combined_classes_t1d_c = pd.Series(df_pep_final_combined_classes_t1d_c)                   

        # sequence
        df_pep_final_sequence_classes_t1d_c = []
        for index, row in df_pep_final_sequence_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pep_final_sequence_classes_t1d_c.append('SP')
            else:
                df_pep_final_sequence_classes_t1d_c.append('Control')
        df_pep_final_sequence_classes_t1d_c = pd.Series(df_pep_final_sequence_classes_t1d_c)                    

        # pro
        df_pro_final_human_classes_t1d_c = []
        for index, row in df_pro_final_human_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pro_final_human_classes_t1d_c.append('SP')
            else:
                df_pro_final_human_classes_t1d_c.append('Control')
        df_pro_final_human_classes_t1d_c = pd.Series(df_pro_final_human_classes_t1d_c)   
        
        df_pro_final_bac_classes_t1d_c = []
        for index, row in df_pro_final_bac_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pro_final_bac_classes_t1d_c.append('SP')
            else:
                df_pro_final_bac_classes_t1d_c.append('Control')
        df_pro_final_bac_classes_t1d_c = pd.Series(df_pro_final_bac_classes_t1d_c)   
                      
        # combined
        df_pro_final_combined_classes_t1d_c = []
        for index, row in df_pro_final_combined_t1d_c.iterrows():
            if str(index).endswith('SP'):
                df_pro_final_combined_classes_t1d_c.append('SP')
            else:
                df_pro_final_combined_classes_t1d_c.append('Control')
        df_pro_final_combined_classes_t1d_c = pd.Series(df_pro_final_combined_classes_t1d_c)
        
        print("Human T1D C has %s samples and %s peptides." % (df_pep_final_human_t1d_c.shape[0], df_pep_final_human_t1d_c.shape[1]))
        print("Bac T1D C has %s samples and %s peptides." % (df_pep_final_bac_t1d_c.shape[0], df_pep_final_bac_t1d_c.shape[1]))       
        print("Combined T1D C has %s samples and %s peptides." % (df_pep_final_combined_t1d_c.shape[0], df_pep_final_combined_t1d_c.shape[1]))       
        print("Sequence-Only T1D C has %s samples and %s peptides." % (df_pep_final_sequence_t1d_c.shape[0], df_pep_final_sequence_t1d_c.shape[1]))   

        print("Human T1D C has %s samples and %s proteins." % (df_pro_final_human_t1d_c.shape[0], df_pro_final_human_t1d_c.shape[1]))
        print("Bac T1D C has %s samples and %s proteins." % (df_pro_final_bac_t1d_c.shape[0], df_pro_final_bac_t1d_c.shape[1]))       
        print("Combined T1D C has %s samples and %s proteins." % (df_pro_final_combined_t1d_c.shape[0], df_pro_final_combined_t1d_c.shape[1]))       
        
        print("##########################################################################")
        print("Peptides")
        print("##########################################################################")
        print("T1D vs C")
        features = [df_pep_final_human_t1d_c, df_pep_final_bac_t1d_c, df_pep_final_combined_t1d_c, df_pep_final_sequence_t1d_c]
        classes = [df_pep_final_human_classes_t1d_c, df_pep_final_bac_classes_t1d_c, df_pep_final_combined_classes_t1d_c, df_pep_final_sequence_classes_t1d_c]
        generate_graph(features, classes, d, "T1D-C", "peptides", False)

        print("##########################################################################")
        print("Proteins")
        print("##########################################################################")
        print("T1D vs C")
        features = [df_pro_final_human_t1d_c, df_pro_final_bac_t1d_c, df_pro_final_combined_t1d_c]
        classes = [df_pro_final_human_classes_t1d_c, df_pro_final_bac_classes_t1d_c, df_pro_final_combined_classes_t1d_c]
        generate_graph(features, classes, d, "T1D-C", "proteins", False)
        
    # here it will be ALL vs AML
    elif d == 'al1':
        p_all_aml = ('ALL', 'AML')  
        df_pep_final_human_all_aml = df_pep_final_human[df_pep_final_human.index.str.endswith(p_all_aml)]
  
        df_pep_final_bac_all_aml = df_pep_final_bac[df_pep_final_bac.index.str.endswith(p_all_aml)]
    
        df_pep_final_combined_all_aml = df_pep_final_combined[df_pep_final_combined.index.str.endswith(p_all_aml)]

        df_pep_final_sequence_all_aml = df_pep_final_sequence[df_pep_final_sequence.index.str.endswith(p_all_aml)]

        # pro
        df_pro_final_human_all_aml = df_pro_final_human[df_pro_final_human.index.str.endswith(p_all_aml)]
  
        df_pro_final_bac_all_aml = df_pro_final_bac[df_pro_final_bac.index.str.endswith(p_all_aml)]
    
        df_pro_final_combined_all_aml = df_pro_final_combined[df_pro_final_combined.index.str.endswith(p_all_aml)]
        
        df_pep_final_human_classes_all_aml = []
        for index, row in df_pep_final_human_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pep_final_human_classes_all_aml.append('ALL')
            else:
                df_pep_final_human_classes_all_aml.append('AML')
        df_pep_final_human_classes_all_aml = pd.Series(df_pep_final_human_classes_all_aml)                    
             
        df_pep_final_bac_classes_all_aml = []
        for index, row in df_pep_final_bac_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pep_final_bac_classes_all_aml.append('ALL')
            else:
                df_pep_final_bac_classes_all_aml.append('AML')
        df_pep_final_bac_classes_all_aml = pd.Series(df_pep_final_bac_classes_all_aml)                   
                      
        # combined
        df_pep_final_combined_classes_all_aml = []
        for index, row in df_pep_final_combined_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pep_final_combined_classes_all_aml.append('ALL')
            else:
                df_pep_final_combined_classes_all_aml.append('AML')
        df_pep_final_combined_classes_all_aml = pd.Series(df_pep_final_combined_classes_all_aml)                  

        # sequence
        df_pep_final_sequence_classes_all_aml = []
        for index, row in df_pep_final_sequence_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pep_final_sequence_classes_all_aml.append('ALL')
            else:
                df_pep_final_sequence_classes_all_aml.append('AML')
        df_pep_final_sequence_classes_all_aml = pd.Series(df_pep_final_sequence_classes_all_aml)                       

        # pro
        df_pro_final_human_classes_all_aml = []
        for index, row in df_pro_final_human_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pro_final_human_classes_all_aml.append('ALL')
            else:
                df_pro_final_human_classes_all_aml.append('AML')
        df_pro_final_human_classes_all_aml = pd.Series(df_pro_final_human_classes_all_aml)                    
             
        df_pro_final_bac_classes_all_aml = []
        for index, row in df_pro_final_bac_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pro_final_bac_classes_all_aml.append('ALL')
            else:
                df_pro_final_bac_classes_all_aml.append('AML')
        df_pro_final_bac_classes_all_aml = pd.Series(df_pro_final_bac_classes_all_aml)                   
                      
        # combined
        df_pro_final_combined_classes_all_aml = []
        for index, row in df_pro_final_combined_all_aml.iterrows():
            if str(index).endswith('ALL'):
                df_pro_final_combined_classes_all_aml.append('ALL')
            else:
                df_pro_final_combined_classes_all_aml.append('AML')
        df_pro_final_combined_classes_all_aml = pd.Series(df_pro_final_combined_classes_all_aml)                  
           
        print("Human ALL AML has %s samples and %s peptides." % (df_pep_final_human_all_aml.shape[0], df_pep_final_human_all_aml.shape[1]))
        print("Bac ALL AML has %s samples and %s peptides." % (df_pep_final_bac_all_aml.shape[0], df_pep_final_bac_all_aml.shape[1]))       
        print("Combined ALL AML has %s samples and %s peptides." % (df_pep_final_combined_all_aml.shape[0], df_pep_final_combined_all_aml.shape[1]))       
        print("Grouped ALL AML has %s samples and %s peptides." % (df_pep_final_sequence_all_aml.shape[0], df_pep_final_sequence_all_aml.shape[1]))   

        print("Human ALL AML has %s samples and %s proteins." % (df_pro_final_human_all_aml.shape[0], df_pro_final_human_all_aml.shape[1]))
        print("Bac ALL AML has %s samples and %s proteins." % (df_pro_final_bac_all_aml.shape[0], df_pro_final_bac_all_aml.shape[1]))       
        print("Combined ALL AML has %s samples and %s proteins." % (df_pro_final_combined_all_aml.shape[0], df_pro_final_combined_all_aml.shape[1]))       
        
        print("##########################################################################")
        print("Peptides")
        print("##########################################################################")
        print("ALL vs AML")
        features = [df_pep_final_human_all_aml, df_pep_final_bac_all_aml, df_pep_final_combined_all_aml, df_pep_final_sequence_all_aml]
        classes = [df_pep_final_human_classes_all_aml, df_pep_final_bac_classes_all_aml, df_pep_final_combined_classes_all_aml, df_pep_final_sequence_classes_all_aml]
        generate_graph(features, classes, d, "ALL-AML", "peptides", False)

        print("##########################################################################")
        print("Proteins")
        print("##########################################################################")
        print("ALL vs AML")
        features = [df_pro_final_human_all_aml, df_pro_final_bac_all_aml, df_pro_final_combined_all_aml]
        classes = [df_pro_final_human_classes_all_aml, df_pro_final_bac_classes_all_aml, df_pro_final_combined_classes_all_aml]
        generate_graph(features, classes, d, "ALL-AML", "proteins", False)
        
print("Number of splits: %s, number of repeats: %s" % (n_splits, n_repeats))
for d in datasets:
    do_ml(d)

    results_pep = pd.DataFrame(rows_list_pep)
    results_pep.to_csv(os.path.join(output_dir, "%s_results_%s_%s.tsv" % (d, n_splits, n_repeats)), sep='\t')
    results_pro = pd.DataFrame(rows_list_pro)
    results_pro.to_csv(os.path.join(output_dir, "%s_results_%s_%s_proteins.tsv" % (d, n_splits, n_repeats)), sep='\t')
    rows_list_pep = []
    rows_list_pro = []


Number of splits: 5, number of repeats: 10
##########################################################################
##########################################################################
Dataset: cd2
Peptides
Human CD C has 122 samples and 600 peptides.
Bac CD C has 122 samples and 42 peptides.
Combined CD C has 122 samples and 642 peptides.
Sequence-Only CD C has 122 samples and 1080 peptides.
---
Human UC C has 113 samples and 600 peptides.
Bac UC C has 113 samples and 42 peptides.
Combined UC C has 113 samples and 642 peptides.
Sequence-Only UC C has 113 samples and 1080 peptides.
---
Human CD UC has 113 samples and 600 peptides.
Bac CD UC has 113 samples and 42 peptides.
Combined CD UC has 113 samples and 642 peptides.
Sequence-Only CD UC has 113 samples and 1080 peptides.
---
Human CD UC C has 174 samples and 600 peptides.
Bac CD UC C has 174 samples and 42 peptides.
Combined CD UC C has 174 samples and 642 peptides.
Sequence-Only CD UC C has 174 samples and 1080 peptides.
P